In [7]:
import sqlite3
from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo=False)
import numpy as np
import pandas as pd

In [2]:
# code to clean up calls.csv and move it to a SQL table



calls = pd.read_csv("data/Call_Data.csv")

def callstime(df, col_arrive):
    yearmonthday = df[col_arrive].astype(str).str.split(" ")
    df["year"] = yearmonthday.apply(lambda x: x[2])
    df["day"] = yearmonthday.apply(lambda x: x[1])
    month_dict = {"Jan": 1, "Feb": 2, "Mar": 3, "Apr": 4, "May": 5, "Jun": 6, "Jul": 7, "Aug": 8, "Sep": 9, "Oct": 10, "Nov": 11, "Dec":12}
    df["month"] = yearmonthday.apply(lambda x: month_dict[x[0]])
    df["time"] = yearmonthday.apply(lambda x: x[3])
    return df

calls = callstime(calls,"Arrived Time")
calls17 = calls[calls["year"].isin(["2014", "2015", "2016", "2017", "2018", "2019"])]

# calculate and add column for epoch time
bar = pd.to_datetime( calls17["Arrived Time"], format="%b %d %Y %I:%M:%S:%f%p" )
epoch_second2 = bar.map(lambda x: x.value/1e9)
calls17["epoch_calls"] = epoch_second2

rs = calls17["CAD Event Number"].value_counts()
two = []
for v, i in zip(rs.values, rs.index):
    if v >1:
        two.append(i)
# len(two) == 362  
st = [x > 1 for x in rs]
dt =rs[st]

t = calls17[calls17["CAD Event Number"].isin(two)]
indt = t[t["Event Clearance Description"] == "-"].index
indt

calls17.drop(indt, axis=0, inplace=True)

#remove repetitive columns
calls_tosql = calls17.drop(columns =["year","day", "month", "time", "Precinct", "Sector", "Arrived Time"])
calls_tosql.set_index("epoch_calls", inplace=True)

# move calls to sqlite table - DONT REPEAT
#calls_tosql.to_sql("log", con=cnx, index=True, if_exists="replace")


/Users/sarahburgart/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [1]:
# exploring calls17 db to doublecheck the ratios are correct in merged db
import numpy as np
import pandas as pd

calls = pd.read_csv("data/Call_Data.csv")

In [2]:
def callstime(df, col_arrive):
    yearmonthday = df[col_arrive].astype(str).str.split(" ")
    df["year"] = yearmonthday.apply(lambda x: x[2])
    df["day"] = yearmonthday.apply(lambda x: x[1])
    month_dict = {"Jan": 1, "Feb": 2, "Mar": 3, "Apr": 4, "May": 5, "Jun": 6, "Jul": 7, "Aug": 8, "Sep": 9, "Oct": 10, "Nov": 11, "Dec":12}
    df["month"] = yearmonthday.apply(lambda x: month_dict[x[0]])
    df["time"] = yearmonthday.apply(lambda x: x[3])
    return df

calls = callstime(calls,"Arrived Time")
calls17 = calls[calls["year"].isin(["2014", "2015", "2016", "2017", "2018", "2019"])]


In [3]:
# calculate and add column for epoch time
bar = pd.to_datetime( calls17["Arrived Time"], format="%b %d %Y %I:%M:%S:%f%p" )
epoch_second2 = bar.map(lambda x: x.value/1e9)
calls17["epoch_calls"] = epoch_second2

rs = calls17["CAD Event Number"].value_counts()
two = []
for v, i in zip(rs.values, rs.index):
    if v >1:
        two.append(i)
# len(two) == 362  
st = [x > 1 for x in rs]
dt =rs[st]

t = calls17[calls17["CAD Event Number"].isin(two)]
indt = t[t["Event Clearance Description"] == "-"].index

/Users/sarahburgart/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [5]:
calls17.drop(indt, axis=0, inplace=True)

#remove repetitive columns
calls_tosql = calls17.drop(columns =["year","day", "month", "time", "Precinct", "Sector", "Arrived Time"])
calls_tosql.set_index("epoch_calls", inplace=True)



KeyError: '[2914744 2931418 2934929 2939226 2941399 2942324 2950082 2951227 2972146\n 2975217 2975263 2975270 2975303 2975316 2975321 2975333 2975339 2975377\n 2975385 2975389 2979903 2980923 2995632 2996678 2996697 2998592 3002153\n 3006563 3015437 3015441 3034528 3034812 3036712 3041149 3046544 3046570\n 3047690 3050611 3065451 3066677 3066691 3067879 3070029 3073577 3075018\n 3080013 3082434 3082732 3084075 3085388 3087801 3088988 3089872 3104396\n 3108138 3108306 3109547 3110825 3111878 3112811 3112875 3121204 3121253\n 3121272 3121281 3124837 3126174 3127259 3127443 3141211 3141223 3154833\n 3155899 3158325 3161196 3162448 3162535 3163719 3164823 3164869 3184500\n 3187171 3191454 3195090 3195140 3197490 3197574 3216168 3218392 3218476\n 3222661 3223776 3224837 3227161 3234465 3239387 3239444 3239490 3252053\n 3252412 3253351 3258127 3260199 3260434 3262770 3265031 3267331 3271091\n 3273255 3273391 3276930 3288694 3293098 3295597 3296850 3298762 3302048\n 3302483 3303960 3304377 33

In [8]:
# create engine
cnx = sqlite3.connect('calls.sqlite')

# create cursor
cur = cnx.cursor()


# move calls to sqlite table - DONT REPEAT
calls_tosql.to_sql("log", con=cnx, index=True, if_exists="replace")

/Users/sarahburgart/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:2531: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  dtype=dtype, method=method)


In [10]:
calls17["Initial Call Type"].value_counts()

PREMISE CHECK, OFFICER INITIATED ONVIEW ONLY        134965
SUSPICIOUS PERSON, VEHICLE OR INCIDENT               85409
TRAFFIC STOP - OFFICER INITIATED ONVIEW              81267
DISTURBANCE, MISCELLANEOUS/OTHER                     71710
OFF DUTY EMPLOYMENT                                  55398
THEFT (DOES NOT INCLUDE SHOPLIFT OR SVCS)            43038
DIRECTED PATROL ACTIVITY                             40320
NUISANCE - MISCHIEF                                  36041
SUSPICIOUS STOP - OFFICER INITIATED ONVIEW           35070
PARKING VIOLATION (EXCEPT ABANDONED CAR)             33522
TRESPASS                                             32495
FOLLOW UP                                            27906
SHOPLIFT - THEFT                                     23003
BURG - RES (INCL UNOCC STRUCTURES ON PROP)           22585
ALARM - RESIDENTIAL - BURGLARY, SILENT/AUDIBLE       21609
ALARM - COMM, SILENT/AUD BURG (INCL BANKS)           20888
NARCOTICS - VIOLATIONS (LOITER, USE, SELL, NARS)     202